In [ ]:
import sys
sys.path.append('..')
import umbrella_mesh
import elastic_rods
import linkage_vis

In [ ]:
import numpy as np
import numpy.linalg as la

### Material

In [ ]:
width = 1
thickness = width / 1.2
cross_section = [thickness, width]

### Initialization

### Lilium

In [ ]:
width = 5
thickness = width * 1
cross_section = [thickness, width]

input_path = '../../data/lilium.json'
from load_jsondata import read_data
vertices, edges, input_alphas, input_ghost_bisectors, input_ghost_normals, A_segments, B_segments, input_segment_normals, input_midpoint_offsets_A, input_midpoint_offsets_B, is_rigid  = read_data(filepath = input_path)


### Unit T shape

In [ ]:
# width = 0.1
# thickness = width / 1.2
# cross_section = [thickness, width]

# vertices = np.array([[0, 0, 0], [0, 1, 0], [1, 0, 0], [-1, 0, 0]])

# edges = np.array([[0, 1], [0, 2], [0, 3]])

# input_alphas = [0, 0, 0, 0]

# input_ghost_bisectors = np.array([[0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0]])

# input_ghost_normals = np.array([[1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0]])

# A_segments = [[0], [0], [1], [2]]
# B_segments = [[1, 2], [], [], []]

# zero_offset = np.array([0, 0, 0])
# special_offset = np.array([0.3, 0.1, 0.1])
# input_midpoint_offsets_A = [[special_offset], [zero_offset], [special_offset], [zero_offset]]
# input_midpoint_offsets_B = [[zero_offset, zero_offset], [], [], []]

# input_segment_normals = np.array([[2., 1, 3], [0, 1, -1], [0, 0, 1]])

# for nid in range(3):
#     input_segment_normals[nid] = input_segment_normals[nid]/np.linalg.norm(input_segment_normals[nid])
# is_rigid = [0, 1, 1, 1]

In [ ]:
curr_um = umbrella_mesh.UmbrellaMesh(vertices, edges, 10, input_alphas, input_ghost_bisectors, input_ghost_normals, A_segments, B_segments, input_midpoint_offsets_A, input_midpoint_offsets_B, input_segment_normals, is_rigid)

In [ ]:
curr_um.setMaterial(elastic_rods.RodMaterial('rectangle', 2000, 0.3, cross_section, stiffAxis=elastic_rods.StiffAxis.D1))

In [ ]:
curr_um.energy()

### Visualization

In [ ]:
view = linkage_vis.LinkageViewer(curr_um, width=1024, height=600)
view.averagedMaterialFrames = True
# view.setCameraParams(camParam)
view.show()

In [ ]:
import py_newton_optimizer
OPTS = py_newton_optimizer.NewtonOptimizerOptions()
OPTS.gradTol = 1e-8
OPTS.verbose = 1;
OPTS.beta = 1e-8
OPTS.niter = 200
OPTS.verboseNonPosDef = False

In [ ]:
# def eqm_callback(prob, i):
#     view.update()

In [ ]:
# umbrella_mesh.compute_equilibrium(curr_um, targetAverageAngle= np.deg2rad(90) - curr_um.averageJointAngle, callback = eqm_callback, options = OPTS, fixedVars = curr_um.rigidJointAngleDoFIndices())

In [ ]:
view.update()

In [ ]:
len(curr_um.rigidJointAngleDoFIndices())

In [ ]:
import importlib, open_umbrella
importlib.reload(open_umbrella)
from open_umbrella import open_umbrella

In [ ]:
from time import sleep
sleep(0.5)
def equilibriumSolver(tgtAngle, um, opts, fv):
    opts.gradTol = 1e-4
    return umbrella_mesh.compute_equilibrium(um, tgtAngle, options=opts, fixedVars=fv)
driver = curr_um.centralJoint()
open_umbrella(curr_um, driver, np.deg2rad(90) - curr_um.averageJointAngle, 50, view, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=100, useTargetAngleConstraint=True, fixedVars = curr_um.rigidJointAngleDoFIndices())